# Build giant tables

In [2]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.stats import iqr
from scipy import interpolate

In [3]:
# Load HUP_implant_dates.xlsx
patients_df = pd.read_excel("../../Data/HUP_implant_dates.xlsx")
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


In [4]:
# Create a mapping between patient ids and the index of the patient in the patients_df dataframe
patient_hup_id_to_index = {}
for i, patient_id in enumerate(patients_df["hup_id"]):
    patient_hup_id_to_index[patient_id] = i

In [5]:
# Initialize an empty list to hold the data
# completed_hup_ids = [160, 172, 141, 145, 157, 161, 138, 142, 151, 171, 175, 187]
completed_hup_ids = [
    160,
    172,
    # 141, # not enough time before first seizure
    145,
    138,
    142,
    151,
    187,
    180,
    184,
    # 192, # incomplete data
    196,  # not enough time before first seizure
    204,  # not enough time before first seizure
    # 165, # incomplete data
    169,  # not enough time after the last seizure
    173,
    150,  # not enough time before first seizure
    # 154, # incomplete data
    # 158, # incomplete data
    # 207,  # not enough time before first seizure
    223,
    # 192,  ## Monday, August 21, 2023 additions this line and below # incomplete data
    196,  # not enough time before first seizure
    204,  # not enough time before first seizure
    177,
    185,
    189,  # not enough time before first seizure
    # 205,  # not enough time before first seizure
    166,
    170,  # not enough time before first seizure
    174,  # not enough time before first seizure
]
# Sort completed_hup_ids in ascending order
completed_hup_ids.sort()

In [6]:
# Only keep the rows in patients_df that correspond to the completed_hup_ids
patients_df = patients_df[patients_df["hup_id"].isin(completed_hup_ids)]
# reset the index of patients_df
patients_df = patients_df.reset_index(drop=True)
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
1,204,HUP204_phaseII,2020-07-29,07:15:00,2020-08-04 00:00:00,82.1
2,196,HUP196_phaseII,2019-08-12,07:15:00,2019-08-21 08:06:00,60.8
3,189,HUP189_phaseII,2019-04-01,07:15:00,2019-04-05 00:00:00,92.7
4,187,HUP187_phaseII,2019-03-11,07:15:00,2019-03-19 00:00:00,86.2
5,185,HUP185_phaseII,2019-01-21,07:15:00,2019-02-01 00:00:00,76.2
6,184,HUP184_phaseII,2019-01-14,07:15:00,2019-01-21 00:00:00,90.7
7,180,HUP180_phaseII,2018-11-05,07:15:00,2018-11-13 00:00:00,51.3
8,177,HUP177_phaseII,2018-08-22,07:15:00,2018-08-31 00:00:00,88.4
9,174,HUP174_phaseII,2018-07-11,07:15:00,2018-07-19 00:00:00,70.6


In [7]:
# Load ./data/ieeg_starts.xlsx into a dataframe
ieeg_starts_df = pd.read_excel("../../Data/ieeg_starts.xlsx")
ieeg_starts_df.tail(10)

,hup_id,ieeg_start
70,146,35.114444
71,145,38.459167
72,144,37.567500
73,143,36.282500
74,142,35.651944
75,141,37.249444
76,140,35.197500
77,139,37.151667
78,138,37.529444
79,137,13.654444


In [7]:
all_med_names = []

for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id = row.hup_id

    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Plot dose curves
    for med_name in all_med_names_plot:
        all_med_names.append(med_name)

all_med_names = np.unique(np.array(all_med_names, dtype=str))
all_med_names

array(['carbamazepine', 'clobazam', 'clonazepam', 'eslicarbazepine',
       'lacosamide', 'lamotrigine', 'levetiracetam', 'lorazepam',
       'oxcarbazepine', 'rufinamide', 'topiramate', 'valproic acid',
       'zonisamide'], dtype='<U15')

In [8]:
# Load aed_ref_ranges.xlsx from ./data/
aed_ref_ranges_df = pd.read_excel("../../Data/aed_ref_ranges.xlsx")
# Lowercase Drug column
aed_ref_ranges_df["Drug"] = aed_ref_ranges_df["Drug"].str.lower()
# show unique units
print(aed_ref_ranges_df["Unit"].unique())
# mg/L and ug/mL are the same
# If Unit is ng/mL, convert to ug/mL
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Min"] = (
    aed_ref_ranges_df["Min"] / 1000
)
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Max"] = (
    aed_ref_ranges_df["Max"] / 1000
)
# Add a column that takes the average of Min and Max
aed_ref_ranges_df["Avg"] = (aed_ref_ranges_df["Min"] + aed_ref_ranges_df["Max"]) / 2
aed_ref_ranges_df

['mg/L' 'ug/mL' 'ng/mL']


,Drug,Min,Max,Unit,Avg
0,levetiracetam,12.00,46.00,mg/L,29.000
1,carbamazepine,4.00,10.00,mg/L,7.000
2,oxcarbazepine,3.00,35.00,ug/mL,19.000
3,clobazam,0.03,0.30,ng/mL,0.165
4,n-desmethylclobazam,0.30,3.00,ng/mL,1.650
5,topiramate,5.00,20.00,mg/L,12.500
6,valproic acid,50.00,125.00,ug/mL,87.500
7,lacosamide,1.00,10.00,ug/mL,5.500
8,felbamate,30.00,60.00,ug/mL,45.000
9,lamotrigine,2.50,15.00,mg/L,8.750


In [9]:
frequency_bands = ["broadband", "60_100", "100_125"]

In [10]:
for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id, patient_weight = row.hup_id, row.weight_kg
    patient_idx = patient_hup_id_to_index[patient_hup_id]
    print(f"Processing HUP {patient_hup_id}")

    # Find the ieeg_start value for patient_hup_id in ieeg_starts_df and convert it into float
    ieeg_start_minutes = (
        float(
            ieeg_starts_df.loc[
                ieeg_starts_df["hup_id"] == patient_hup_id, "ieeg_start"
            ].values[0]
        )
        * 60
    )

    ##############################################
    # MEDICATIONS
    ##############################################

    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Construct the time axis
    emu_start_time_hrs = min([all_tHr_plot[i][0] for i in range(len(all_tHr_plot))])
    emu_end_time_hrs = all_tHr_plot[0][-1]
    max_dose_duration_hrs = emu_end_time_hrs - emu_start_time_hrs
    max_length = max([len(all_tHr_plot[i]) for i in range(len(all_tHr_plot))])
    time_axis = np.linspace(emu_start_time_hrs, emu_end_time_hrs, max_length)

    # Create a dataframe that will hold the dose curves for all patients
    hourly_patient_features_df = pd.DataFrame(columns=["emu_time"])
    hourly_patient_features_df["emu_time"] = time_axis

    for potential_med_name in all_med_names:
        hourly_patient_features_df[f"med_{potential_med_name}_raw"] = np.zeros(
            len(time_axis)
        )

    sum_array = []

    ##############################################
    # MEDICATIONS Normalize to 1
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()
        dose = all_dose_curves_plot[med_idx].flatten()

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)
    cumulative_dose_curve = cumulative_dose_curve / np.max(cumulative_dose_curve)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_raw"] = cumulative_dose_curve

    ##############################################
    # MEDICATIONS Normalize with DDD
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()

        # Find Avg for medication med_name in aed_ref_ranges_df
        if med_name != "lorazepam":
            ref_range = float(
                aed_ref_ranges_df.loc[
                    aed_ref_ranges_df["Drug"] == med_name, "Avg"
                ].values[0]
            )
        else:
            ref_range = 1
        print(f"{med_name} has ref range {ref_range} mg/L")

        dose = all_dose_curves_plot[med_idx].flatten()
        dose = dose / ref_range

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_ddd"] = cumulative_dose_curve

    ##############################################
    # Group by 2 minutes and compute mean
    ##############################################
    hourly_patient_features_df["emu_minute"] = (
        (hourly_patient_features_df["emu_time"] * 60).astype(int) // 2 * 2
    )
    hourly_patient_features_df = hourly_patient_features_df.groupby("emu_minute").mean()
    hourly_patient_features_df = hourly_patient_features_df.reset_index()
    hourly_patient_features_df = hourly_patient_features_df.drop(columns=["emu_time"])

    ##############################################
    # SEIZURE COUNT
    ##############################################
    seizure_times_sec = np.load(
        f"../../Data/seizures/source_mat/HUP_{patient_hup_id}.npy"
    )
    seizure_times_sec = seizure_times_sec + (ieeg_start_minutes * 60)

    # Convert seizure times from seconds to minutes
    seizure_times_min = seizure_times_sec / 60

    hourly_patient_features_df["had_seizure"] = np.zeros(
        len(hourly_patient_features_df), dtype=int
    )

    for sz_min in seizure_times_min[:, 0]:
        hourly_patient_features_df.loc[
            hourly_patient_features_df["emu_minute"] == int(sz_min) // 2 * 2,
            "had_seizure",
        ] += 1

    ##############################################
    # Time since last seizure
    ##############################################

    # Initialize the list and timer
    time_since_last_seizure = []
    timer = None

    # Loop through the dataframe and calculate the time since the last seizure
    for had_seizure in hourly_patient_features_df["had_seizure"]:
        if had_seizure == 1:
            timer = 0
        elif timer is not None:  # if there has been a seizure before
            timer += 2
        else:
            timer = None
        time_since_last_seizure.append(timer)

    # Add the list as a new column
    hourly_patient_features_df["time_since_last_seizure"] = time_since_last_seizure

    ##############################################
    # Time since first seizure
    ##############################################
    # Initialize the list and timer
    time_since_first_seizure = []
    timer = None
    first_seizure_occurred = False

    # Loop through the dataframe and calculate the time since the first seizure
    for had_seizure in hourly_patient_features_df["had_seizure"]:
        if had_seizure == 1 and not first_seizure_occurred:
            timer = 0
            first_seizure_occurred = True
        elif first_seizure_occurred:  # if the first seizure has already happened
            timer += 2
        else:
            timer = None
        time_since_first_seizure.append(timer)

    # Add the list as a new column
    hourly_patient_features_df["time_since_first_seizure"] = time_since_first_seizure

    ##########################################
    # SYNCHRONY
    ##########################################

    # Determine the starting index for the synchrony data
    start_index = None
    for i, emu_min in enumerate(hourly_patient_features_df["emu_minute"]):
        if i < len(hourly_patient_features_df["emu_minute"]) - 1:
            next_emu_min = hourly_patient_features_df["emu_minute"].iloc[i + 1]
        else:
            next_emu_min = emu_min + 2

        if emu_min <= ieeg_start_minutes < next_emu_min:
            start_index = i
            break

    if start_index is None:
        raise ValueError(
            "Could not determine starting index for synchrony data based on ieeg_start_minutes value"
        )

    for frequency_band in frequency_bands:
        synchrony_np = np.load(
            f"../../Data/synchrony/all/{frequency_band}/HUP_{patient_hup_id}.npy"
        )

        # Initialize the synchrony column with NaNs
        hourly_patient_features_df[f"synchrony_{frequency_band}"] = np.nan

        # Insert synchrony values starting from the appropriate index
        end_index = min(
            start_index + len(synchrony_np), len(hourly_patient_features_df)
        )
        hourly_patient_features_df.iloc[
            start_index:end_index,
            hourly_patient_features_df.columns.get_loc(f"synchrony_{frequency_band}"),
        ] = synchrony_np[: end_index - start_index]

    ##########################################
    # AD Ratio
    ##########################################
    mat_file = scipy.io.loadmat(
        f"../../../erinconr/projects/fc_toolbox/results/analysis/intermediate/HUP{patient_hup_id}.mat"
    )
    mat_file = mat_file["summ"][0][0]
    ad_ratio = mat_file[17]
    num_channels = mat_file[6].shape[0]
    assert ad_ratio.shape[0] == num_channels

    ad_ratio = np.nanmean(ad_ratio, axis=0)
    ad_ratio = (ad_ratio - np.nanmedian(ad_ratio)) / iqr(ad_ratio, nan_policy="omit")
    assert np.nansum(ad_ratio) != 0

    # Reshape ad_ratio to match the granularity of the dataframe
    reshaped_ad_ratio = np.repeat(ad_ratio, 5)

    # Initialize the ad_ratio column with NaNs
    hourly_patient_features_df["ad_ratio"] = np.nan

    # Insert reshaped_ad_ratio values starting from the appropriate index
    end_index_ad_ratio = min(
        start_index + len(reshaped_ad_ratio), len(hourly_patient_features_df)
    )
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("ad_ratio"),
    ] = reshaped_ad_ratio[: end_index_ad_ratio - start_index]

    ##########################################
    # EEG time
    ##########################################
    # Create the eeg_time column with NaN values
    hourly_patient_features_df["eeg_time"] = np.nan

    # Define the eeg_time values starting from start_index
    eeg_time_values = np.arange(
        0, (end_index_ad_ratio - start_index) * 2, 2
    )  # incrementing by 2 since the time is grouped by 2 minutes
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("eeg_time"),
    ] = eeg_time_values

    ##############################################
    # SAVE TO CSV
    ##############################################

    hourly_patient_features_df.to_csv(
        f"../../Data/giant_new_tables/HUP_{patient_hup_id}.csv", index=False
    )

Processing HUP 223
lacosamide has ref range 5.5 mg/L
levetiracetam has ref range 29.0 mg/L
lorazepam has ref range 1 mg/L


/tmp/ipykernel_633/4139959151.py:235: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 204
clobazam has ref range 0.16499999999999998 mg/L
clonazepam has ref range 0.045000000000000005 mg/L
lacosamide has ref range 5.5 mg/L
lorazepam has ref range 1 mg/L
valproic acid has ref range 87.5 mg/L


/tmp/ipykernel_633/4139959151.py:235: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 196
lamotrigine has ref range 8.75 mg/L
levetiracetam has ref range 29.0 mg/L
lorazepam has ref range 1 mg/L


...

/tmp/ipykernel_633/4139959151.py:235: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 142
carbamazepine has ref range 7.0 mg/L
levetiracetam has ref range 29.0 mg/L


/tmp/ipykernel_633/4139959151.py:235: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 138
lacosamide has ref range 5.5 mg/L
levetiracetam has ref range 29.0 mg/L
lorazepam has ref range 1 mg/L


/tmp/ipykernel_633/4139959151.py:235: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


In [11]:
hourly_patient_features_df

,emu_minute,med_carbamazepine_raw,med_clobazam_raw,med_clonazepam_raw,med_eslicarbazepine_raw,med_lacosamide_raw,med_lamotrigine_raw,med_levetiracetam_raw,med_lorazepam_raw,med_oxcarbazepine_raw,...,med_sum_no_lorazepam_raw,med_sum_no_lorazepam_ddd,had_seizure,time_since_last_seizure,time_since_first_seizure,synchrony_broadband,synchrony_60_100,synchrony_100_125,ad_ratio,eeg_time
0,1362,0.0,0.0,0.0,0.0,0.000000,0.0,0.341401,0.000000,0.0,...,0.279076,10.242039,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1364,0.0,0.0,0.0,0.0,0.000000,0.0,0.400841,0.000000,0.0,...,0.327665,12.025243,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1366,0.0,0.0,0.0,0.0,0.000000,0.0,0.472000,0.000000,0.0,...,0.385834,14.160009,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1368,0.0,0.0,0.0,0.0,0.000000,0.0,0.534444,0.000000,0.0,...,0.436878,16.033327,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1370,0.0,0.0,0.0,0.0,0.000000,0.0,0.589194,0.000000,0.0,...,0.481633,17.675825,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,11400,0.0,0.0,0.0,0.0,1.228754,0.0,0.880147,0.009945,0.0,...,0.909968,34.391318,0,1508.0,5008.0,0.954573,0.117115,0.151084,-0.08901,9150.0
5020,11402,0.0,0.0,0.0,0.0,1.227028,0.0,0.878269,0.009926,0.0,...,0.908165,34.323765,0,1510.0,5010.0,0.956238,0.114422,0.135446,-0.08901,9152.0
5021,11404,0.0,0.0,0.0,0.0,1.225304,0.0,0.876266,0.009906,0.0,...,0.906260,34.252466,0,1512.0,5012.0,0.955778,0.109358,0.159385,-0.08901,9154.0
5022,11406,0.0,0.0,0.0,0.0,1.223582,0.0,0.874154,0.009887,0.0,...,0.904267,34.177909,0,1514.0,5014.0,0.956902,0.105903,0.122097,-0.08901,9156.0
